# Module Leader Menu (IPyWidgets GUI)

This notebook provides a **Jupyter-friendly GUI menu** (IPyWidgets only) to:

1. Import CSVs into a SQLite database (`CWDatabase.db`)
2. View **all results for a student** (across all assessment tables)
3. Run **per-question student performance** analysis vs class average
4. Generate an **underperforming students** report

> **Tip:** Run each cell from top to bottom. After importing CSVs, refresh the table list in the GUI.


In [ ]:
# --- Notebook setup ---
%matplotlib inline

import os
import glob
import traceback
import importlib.util
import ipywidgets as widgets
from IPython.display import display, clear_output


## 1) Configure your project file paths

These point to the `.py` files you attached.  
If you moved them elsewhere, update the paths below.


In [ ]:
# --- paths to your modules (edit if needed) ---
CWPREPROCESSING_PATH = "/mnt/data/CWpreprocessing.py"
TESTRESULTS_PATH     = "/mnt/data/testresults.py"
STUDPERF_PATH        = "/mnt/data/studentpreformance.py"
UNDERPERF_PATH       = "/mnt/data/underpreforming_student.py"

def import_from_path(module_name: str, file_path: str):
    if not os.path.exists(file_path):
        raise FileNotFoundError(f"Module not found at: {file_path}")
    spec = importlib.util.spec_from_file_location(module_name, file_path)
    mod = importlib.util.module_from_spec(spec)
    spec.loader.exec_module(mod)  # type: ignore
    return mod

cwprep_mod   = import_from_path("cwpreprocessing", CWPREPROCESSING_PATH)
test_mod     = import_from_path("testresults", TESTRESULTS_PATH)
studperf_mod = import_from_path("studentpreformance", STUDPERF_PATH)
under_mod    = import_from_path("underpreforming_student", UNDERPERF_PATH)

CSVtoSQLite            = cwprep_mod.CSVtoSQLite
TestResultsAnalyzer    = test_mod.TestResultsAnalyzer
StudentPerformance     = studperf_mod.StudentPerformance
UnderperformingStudents= under_mod.UnderperformingStudents

print("✅ Modules loaded.")


## 2) Launch the GUI menu

Paste CSV paths or choose a folder, then use the tabs.


In [ ]:
# =========================
# GUI MENU (IPyWidgets only)
# =========================

out = widgets.Output(layout={"border": "1px solid #ddd", "padding": "10px"})

db_path = widgets.Text(
    value="CWDatabase.db",
    description="DB path:",
    placeholder="e.g. CWDatabase.db or /path/to/CWDatabase.db",
    layout=widgets.Layout(width="80%")
)

status = widgets.HTML(value="<b>Status:</b> Ready")

def run_safely(fn):
    with out:
        clear_output()
        try:
            fn()
        except Exception as e:
            print("❌ Error:")
            print(e)
            print("\n--- Traceback ---")
            traceback.print_exc()

# ---------- TAB 1: CSV -> SQLite ----------
csv_folder = widgets.Text(
    value="",
    description="CSV folder:",
    placeholder="Paste folder path containing CSVs (optional)",
    layout=widgets.Layout(width="80%")
)

csv_files = widgets.Textarea(
    value="",
    description="CSV files:",
    placeholder="Paste ONE CSV path per line (optional). If provided, these override the folder.",
    layout=widgets.Layout(width="80%", height="110px")
)

convert_btn = widgets.Button(description="Convert CSV(s) → SQLite", button_style="success", icon="database")
show_found  = widgets.Button(description="Preview CSVs found", icon="search")

def resolve_csv_list():
    lines = [ln.strip() for ln in csv_files.value.splitlines() if ln.strip()]
    if lines:
        return lines

    folder = csv_folder.value.strip()
    if not folder:
        raise ValueError("Provide either a CSV folder OR a list of CSV file paths.")

    if not os.path.isdir(folder):
        raise FileNotFoundError(f"Folder not found: {folder}")

    found = sorted(glob.glob(os.path.join(folder, "*.csv")))
    if not found:
        raise FileNotFoundError(f"No CSV files found in: {folder}")
    return found

def on_preview_csvs(_=None):
    def _do():
        paths = resolve_csv_list()
        print(f"Found {len(paths)} CSV file(s):")
        for p in paths:
            print(" -", p)
    run_safely(_do)

def on_convert(_=None):
    def _do():
        paths = resolve_csv_list()
        print(f"Converting {len(paths)} CSV file(s) into SQLite DB (converter default): CWDatabase.db")
        print("If you changed DB path above, note your converter may still write to CWDatabase.db unless edited.\n")

        for p in paths:
            print(f"➡ Converting: {p}")
            conv = CSVtoSQLite(p)
            conv.convert()
        print("\n✅ Done. Database created/updated: CWDatabase.db")
        status.value = "<b>Status:</b> Import complete. Refresh table list in analysis tabs if needed."
    run_safely(_do)

show_found.on_click(on_preview_csvs)
convert_btn.on_click(on_convert)

tab1 = widgets.VBox([
    widgets.HTML("<h3>1) Import CSV files into SQLite</h3>"),
    widgets.HTML("Provide either a <b>folder</b> OR a list of <b>CSV file paths</b>."),
    csv_folder,
    csv_files,
    widgets.HBox([show_found, convert_btn]),
])

# ---------- TAB 2: Student results across ALL tables ----------
student_id_all = widgets.Text(value="", description="Student ID:", placeholder="e.g. 12345", layout=widgets.Layout(width="50%"))
fetch_all_btn = widgets.Button(description="Fetch + Plot Student Results", button_style="primary", icon="chart-bar")

def on_fetch_all(_=None):
    def _do():
        sid = student_id_all.value.strip()
        if not sid:
            raise ValueError("Enter a Student ID.")
        df = TestResultsAnalyzer.get_student_scores(db_path.value.strip(), sid)
        if df is None or df.empty:
            print("No rows returned for this student.")
            return
        display(df)
        TestResultsAnalyzer.plot_student_scores(df, sid)
        print("✅ Done.")
    run_safely(_do)

fetch_all_btn.on_click(on_fetch_all)

tab2 = widgets.VBox([
    widgets.HTML("<h3>2) View all assessment results for one student</h3>"),
    db_path,
    student_id_all,
    fetch_all_btn,
])

# ---------- TAB 3: Per-question performance ----------
sp = None  # will be created when refreshed

refresh_tables_btn = widgets.Button(description="Refresh Table List", icon="refresh")
table_dropdown = widgets.Dropdown(options=[], description="Table:", layout=widgets.Layout(width="80%"))
student_id_q = widgets.Text(value="", description="Student ID:", placeholder="e.g. 12345", layout=widgets.Layout(width="50%"))
analyse_btn = widgets.Button(description="Analyse + Plot (Per Question)", button_style="primary", icon="line-chart")

def refresh_tables(_=None):
    def _do():
        global sp
        sp = StudentPerformance(db_path.value.strip())
        tables = sp.list_tables()
        if not tables:
            table_dropdown.options = []
            table_dropdown.value = None
            print("No tables found. Import CSVs first.")
            return
        default = sp.pick_default_test_table(tables)
        table_dropdown.options = tables
        table_dropdown.value = default if default in tables else tables[0]
        print("✅ Table list refreshed.")
        print("Selected:", table_dropdown.value)
    run_safely(_do)

def on_analyse_question_level(_=None):
    def _do():
        if sp is None:
            raise ValueError("Click 'Refresh Table List' first.")
        sid = student_id_q.value.strip()
        if not sid:
            raise ValueError("Enter a Student ID.")
        table = table_dropdown.value
        if not table:
            raise ValueError("No table selected.")
        result = sp.analyse(sid, table)
        display(result)
        sp.plot(result, sid, table)
        print("✅ Done.")
    run_safely(_do)

refresh_tables_btn.on_click(refresh_tables)
analyse_btn.on_click(on_analyse_question_level)

tab3 = widgets.VBox([
    widgets.HTML("<h3>3) Student performance per question (vs class average)</h3>"),
    db_path,
    widgets.HBox([refresh_tables_btn, widgets.HTML("<span style='color:#666'>Refresh after importing CSVs or changing DB path.</span>")]),
    table_dropdown,
    student_id_q,
    analyse_btn
])

# ---------- TAB 4: Underperforming students report ----------
threshold = widgets.FloatSlider(
    value=40.0, min=0.0, max=100.0, step=1.0,
    description="Threshold:",
    readout_format=".0f",
    layout=widgets.Layout(width="80%")
)

summative_table = widgets.Text(
    value="",
    description="Summative table:",
    placeholder="Optional. Leave blank for auto-detect.",
    layout=widgets.Layout(width="80%")
)

build_btn = widgets.Button(description="Build Report + Plot", button_style="warning", icon="exclamation-triangle")

def on_build_report(_=None):
    def _do():
        ups = UnderperformingStudents(db_path.value.strip())
        summ = summative_table.value.strip() or None
        report, used_summ = ups.build_report(summative_table=summ, threshold=float(threshold.value))
        display(report)
        ups.plot_underperformers(report, used_summ, float(threshold.value))
        print(f"✅ Done. Summative table used: {used_summ}")
    run_safely(_do)

build_btn.on_click(on_build_report)

tab4 = widgets.VBox([
    widgets.HTML("<h3>4) Underperforming students (sorted by summative score)</h3>"),
    db_path,
    summative_table,
    threshold,
    build_btn
])

# ---------- Assemble ----------
tabs = widgets.Tab(children=[tab1, tab2, tab3, tab4, out])
tabs.set_title(0, "CSV → SQLite")
tabs.set_title(1, "Student Results")
tabs.set_title(2, "Per-Question Analysis")
tabs.set_title(3, "Underperformers")
tabs.set_title(4, "Output / Logs")

display(widgets.VBox([
    widgets.HTML("<h2>Module Leader Menu (CW Database Toolkit)</h2>"),
    status,
    tabs
]))

# Optional: pre-refresh (won't crash hard)
with out:
    try:
        refresh_tables()
    except Exception:
        print("ℹ️ Ready. Import CSVs first, then refresh tables in the analysis tab.")
